In [6]:
def gf16v_mul_256(a: int, b: int) -> int:
    """
    Multiplies a (256-bit) 'a' by a 4-bit 'b' under GF(16).
    
    :param a: 256-bit integer (Python int).
    :param b: 4-bit integer (0 <= b <= 0xF).
    :return:  256-bit integer result of the GF(16) multiply.
    """
    # For 256-bit, extend the "0x8 nibble" pattern across 64 hex digits.
    mask_msb = 0x8888888888888888888888888888888888888888888888888888888888888888

    # Ensure 'a' is in 256-bit range; ensure 'b' is in 4-bit range.
    a &= (1 << 256) - 1
    b &= 0xF

    result = 0

    # We have 4 bits in 'b', so iterate for i in {0,1,2,3}.
    for i in range(4):
        # If the i-th bit of b is set, XOR 'a' into result.
        if (b >> i) & 1:
            result ^= a

        # The standard GF(16) step:
        #  1) Extract top bits in each nibble (where bit 3 is set).
        a_msb = a & mask_msb
        #  2) Clear them from 'a'.
        a ^= a_msb
        #  3) Shift left by 1 (multiply by x).
        a <<= 1
        #  4) For all nibble MSBs that overflowed, shift them down by 3 bits
        #     to align with the nibble's LSB, then multiply by 3 and XOR.
        a ^= (a_msb >> 3) * 3

        # If you want to strictly stay within 256 bits, mask again:
        a &= (1 << 256) - 1

    # Mask result as well, if you want a strict 256-bit result.
    return result & ((1 << 256) - 1)




In [10]:
def main():
    # Example usage:
    # 256-bit 'a', with a sample big hex value:
    a_256 = 0xffffffffeeeeeeeeddddddddccccccccbbbbbbbbaaaaaaaa1111111100000000
    # 4-bit 'b' in GF(16):
    b_4   = 0x2

    # for i in range(0x10):
    product = gf16v_mul_256(a_256, b_4)
    # print(f"a   = 0x{a_256:064x}")
    # print(f"b  = 0x{b_4:x}")
    print(f"w0 = 0x{product:064x}")

if __name__ == "__main__":
    main()

w0 = 0xddddddddffffffff99999999bbbbbbbb55555555777777772222222200000000


In [ ]:
# The four blocks we identified
block1 = 0x0F0E0D0C0B0A090807060504030201000F0E0D0C0B0A09080706050403020100
block2 = 0x1111111100000000ffffffffeeeeeeeeddddddddccccccccbbbbbbbbaaaaaaaa
block3 = 0x9999999988888888777777776666666655555555444444443333333322222222
block4 = 0xffffffffeeeeeeeeddddddddccccccccbbbbbbbbaaaaaaaa1111111100000000

block4 = 0x0d0f090b050701030e0c0a08060402000d0f090b050701030e0c0a0806040200
block5 = 0x2222222200000000ddddddddffffffff99999999bbbbbbbb5555555577777777
block6 = 0x1111111133333333eeeeeeeeccccccccaaaaaaaa888888886666666644444444
block7 = 0xddddddddffffffff99999999bbbbbbbb55555555777777772222222200000000
# XOR all blocks together
# result = block1 ^ block2 ^ block3 ^ block4
result = block4 ^ block5 ^ block6 ^ block7
# Print result in hexadecimal format
print(f"Result: {hex(result)}")

Result: 0x78797a7b6d6c6f6e52535051474645443c3d3e3f29282b2a9e9f9c9d8b8a8988
